# RAG ChatBot Project Summary

이 노트북은 프로젝트의 **핵심 기능(검색 및 생성 프로세스)**을 요약하여 빠르게 확인하기 위해 작성되었습니다.
복잡한 웹 UI(`app.py`)를 실행하지 않고 로직이 잘 동작하는지 검증할 수 있습니다.

In [ ]:
import sys
import os

# 프로젝트 루트 경로 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.common.config import config
from src.generation.rag import RAGChain
from src.indexing.vector_store import VectorStoreWrapper

## 1. 시스템 초기화
설정을 로드하고 Vector DB와 RAG 체인을 초기화합니다.

In [ ]:
print(f"🔹 Loading Configuration: {config['model']['llm_name']}")

# Vector Store Load
vector_store = VectorStoreWrapper(config)
vector_store.initialize()

# RAG Chain Load
rag_chain = RAGChain(config=config, vector_store_wrapper=vector_store)

print("✅ System Initialized Successfully!")

## 2. 문서 검색 테스트 (High Quality vs Fast Mode)
Hybrid Search와 Re-ranking이 적용된 검색 결과를 확인합니다.

In [ ]:
query = "이 사업의 예산은 얼마인가요?"

# 1. 일반 모드 (Re-ranking 적용, 정확도 높음)
print(f"\n🔎 Query: {query}")
print("\n--- [Standard Mode] (High Accuracy) ---")
standard_retriever = rag_chain.get_retriever(fast_mode=False)
docs_standard = standard_retriever.invoke(query)
for i, doc in enumerate(docs_standard):
    print(f"[{i+1}] {doc.page_content[:100]}... (Source: {doc.metadata.get('source', 'Unknown')})")

# 2. 고속 모드 (Re-ranking 미적용, 속도 빠름)
print("\n--- [Fast Mode] (High Speed) ---")
fast_retriever = rag_chain.get_retriever(fast_mode=True)
docs_fast = fast_retriever.invoke(query)
for i, doc in enumerate(docs_fast):
    print(f"[{i+1}] {doc.page_content[:100]}... (Source: {doc.metadata.get('source', 'Unknown')})")

## 3. 답변 생성 테스트
`stream_answer` 메서드를 통해 토큰 단위로 생성되는 답변을 시뮬레이션합니다.

In [ ]:
print("\n🤖 Generating Answer (Streaming)...\n")

# 답변 생성
stream = rag_chain.stream_answer(query, docs_standard, level="요약")

full_response = ""
for chunk in stream:
    print(chunk, end="", flush=True)
    full_response += chunk

print("\n\n✅ Generation Complete!")